# <font color="green"> Problem 1 :  条件変数を使った飽和カウンタ</font>

* 下掲のプログラムを変更して, 任意のスレッド数, inc (dec) 回数, capacity で正しく動作するようにせよ
* 寝ているスレッドを起こすのに `pthread_cond_broadcast` を使うべきか, `pthread_cond_signal` でもよいか考えよ
  * ヒント: 実は後者ではダメ. なかなか気づきにくい落とし穴がある. Terminal + GDBで調査でもしないとなかなか気づかないだろう
  * 止まってしまったプログラムをkillする方法は上述したのでそれをマスターしてから実験せよ

In [ ]:
%%writefile scounter_ans.c
/* 注: このプログラムはOMP_NUM_THREADSを使わずにコマンドラインで受け取った引数でスレッド数を決めている(#pragma omp parallel num_threads(...)) */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <omp.h>

void die(char * msg) {
  perror(msg);
  exit(1);
}

double cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

/* 飽和カウンタ */
typedef struct {
  long x;
  long capacity;
  pthread_mutex_t m[1];
  pthread_cond_t c[1];
} scounter_t;

/* 初期化(値を0にする) */
void scounter_init(scounter_t * s, long capacity) {
  s->x = 0;
  s->capacity = capacity;
  if (pthread_mutex_init(s->m, 0)) {
    die("pthread_mutex_init");
  }
  if (pthread_cond_init(s->c, 0)) {
    die("pthread_cond_init");
  }
}

/* +1 ただしcapacityに達していたら待つ */
long scounter_inc(scounter_t * s) {
  pthread_mutex_lock(s->m);
  long x = s->x;
  while (x >= s->capacity) {
    assert(x == s->capacity);
    pthread_cond_wait(s->c, s->m);
    x = s->x;
  }
  s->x = x + 1;
  pthread_mutex_unlock(s->m);
  assert(x < s->capacity);
  return x;
}

/* -1 */
long scounter_dec(scounter_t * s) {
  pthread_mutex_lock(s->m);
  long x = s->x;
  s->x = x - 1;
  if (x >= s->capacity) {
    assert(x == s->capacity);
    pthread_cond_broadcast(s->c);
  }
  pthread_mutex_unlock(s->m);
  return x;
}

/* 現在の値を返す */
long scounter_get(scounter_t * s) {
  return s->x;
}

int main(int argc, char ** argv) {
  int i = 1;
  /* incを呼ぶスレッド数 */
  int n_inc_threads = (argc > i ? atoi(argv[i]) : 3); i++;
  /* decを呼ぶスレッド数 */
  int n_dec_threads = (argc > i ? atoi(argv[i]) : 2); i++;
  /* incとdecが呼ばれる回数(全スレッドの合計) */
  long n            = (argc > i ? atol(argv[i]) : 10000); i++;
  /* 飽和する値 */
  long capacity     = (argc > i ? atol(argv[i]) : 10000); i++;
  
  scounter_t s[1];
  scounter_init(s, capacity);

  printf("increment threads : %d\n", n_inc_threads);
  printf("decrement threads : %d\n", n_dec_threads);
  printf("increments/decrements : %ld\n", n);
  printf("capacity : %ld\n", capacity);
  
  double t0 = cur_time();
#pragma omp parallel num_threads(n_inc_threads + n_dec_threads)
  {
    int idx = omp_get_thread_num();
    if (idx < n_inc_threads) {
      /* increment */
      long a = n *  idx      / n_inc_threads;
      long b = n * (idx + 1) / n_inc_threads;
      for (long i = a; i < b; i++) {
        long x = scounter_inc(s);
        assert(x < capacity);
      }
    } else {
      /* decrement */
      idx -= n_dec_threads;
      long a = n *  idx      / n_dec_threads;
      long b = n * (idx + 1) / n_dec_threads;
      for (long i = a; i < b; i++) {
        long x = scounter_dec(s);
        assert(x <= capacity);
      }
    }
  }
  double t1 = cur_time();
  printf("took %.9f sec\n", t1 - t0);
  long x = scounter_get(s);
  printf("%s : value at the end = %ld\n", (x == 0? "OK" : "NG"), x);
  return (x == 0 ? 0 : 1);
}

In [ ]:
gcc -Wall -fopenmp -o scounter_ans scounter_ans.c

* 以下でテストせよ
  * テストは, 「単純(超探しやすい)」かつ「バグが発生しやすいケース」で行うのが基本
  * 今回の場合, 容量1というケースが最もバグが発生しやすい(見逃されにくい)と思われる

* 単純かつバグが発生しやすいケース (うまく動かなければ適宜 100000 を減らして, なるべく小さい数でテストをするのが基本. 本気でデバッグが必要なら terminal を使うことを推奨)

In [ ]:
./scounter_ans 1 1 100000 1

* まともな容量でのテスト

In [ ]:
./scounter_ans 1 1 100000 1000

* 多数のスレッドでのテスト

In [ ]:
./scounter_ans 10 1 100000 1000

In [ ]:
./scounter_ans 1 10 100000 1000

In [ ]:
./scounter_ans 10 10 1000000 1000

# <font color="green"> Problem 2 :  両側飽和カウンタ</font>

* どうせならカウンタの上限を定めるだけでなく, 下限も定めたい. ここでは0ということにしておこう
* つまり, 0のときに dec が行われたらそこでも待つようにする
* 下掲のプログラムを変更して, 任意のスレッド数, inc (dec) 回数, capacity で正しく動作するようにせよ
* ヒント: decをしようとして値が0だった際に待つための条件変数をもう一つ用意する

In [ ]:
%%writefile tcounter_ans.c
/* 注: このプログラムはOMP_NUM_THREADSを使わずにコマンドラインで受け取った引数でスレッド数を決めている(#pragma omp parallel num_threads(...)) */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <omp.h>

void die(char * msg) {
  perror(msg);
  exit(1);
}

double cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec + ts->tv_nsec * 1.0e-9;
}

/* 飽和カウンタ */
typedef struct {
  long x;
  long capacity;
  pthread_mutex_t m[1];
  pthread_cond_t c[1];
  pthread_cond_t d[1];
} scounter_t;

/* 初期化(値を0にする) */
void scounter_init(scounter_t * s, long capacity) {
  s->x = 0;
  s->capacity = capacity;
  if (pthread_mutex_init(s->m, 0)) {
    die("pthread_mutex_init");
  }
  if (pthread_cond_init(s->c, 0)) {
    die("pthread_cond_init");
  }
  if (pthread_cond_init(s->d, 0)) {
    die("pthread_cond_init");
  }
}

/* +1 ただしcapacityに達していたら待つ */
long scounter_inc(scounter_t * s) {
  pthread_mutex_lock(s->m);
  long x = s->x;
  while (x >= s->capacity) {
    assert(x == s->capacity);
    pthread_cond_wait(s->c, s->m);
    x = s->x;
  }
  s->x = x + 1;
  if (x <= 0) {
    assert(x == 0);
    pthread_cond_broadcast(s->d);
  }
  pthread_mutex_unlock(s->m);
  assert(x < s->capacity);
  return x;
}

/* -1 */
long scounter_dec(scounter_t * s) {
  pthread_mutex_lock(s->m);
  long x = s->x;
  while (x <= 0) {
    assert(x == 0);
    pthread_cond_wait(s->d, s->m);
    x = s->x;
  }
  s->x = x - 1;
  if (x >= s->capacity) {
    assert(x == s->capacity);
    pthread_cond_broadcast(s->c);
  }
  pthread_mutex_unlock(s->m);
  return x;
}

/* 現在の値を返す */
long scounter_get(scounter_t * s) {
  return s->x;
}

int main(int argc, char ** argv) {
  int i = 1;
  /* incを呼ぶスレッド数 */
  int n_inc_threads = (argc > i ? atoi(argv[i]) : 3); i++;
  /* decを呼ぶスレッド数 */
  int n_dec_threads = (argc > i ? atoi(argv[i]) : 2); i++;
  /* incとdecが呼ばれる回数(全スレッドの合計) */
  long n            = (argc > i ? atol(argv[i]) : 10000); i++;
  /* 飽和する値 */
  long capacity     = (argc > i ? atol(argv[i]) : 10000); i++;
  
  scounter_t s[1];
  scounter_init(s, capacity);

  printf("increment threads : %d\n", n_inc_threads);
  printf("decrement threads : %d\n", n_dec_threads);
  printf("increments/decrements : %ld\n", n);
  printf("capacity : %ld\n", capacity);
  
  double t0 = cur_time();
#pragma omp parallel num_threads(n_inc_threads + n_dec_threads)
  {
    int idx = omp_get_thread_num();
    if (idx < n_inc_threads) {
      /* increment */
      long a = n *  idx      / n_inc_threads;
      long b = n * (idx + 1) / n_inc_threads;
      for (long i = a; i < b; i++) {
        long x = scounter_inc(s);
        assert(x < capacity);
        assert(x >= 0);
      }
    } else {
      /* decrement */
      idx -= n_dec_threads;
      long a = n *  idx      / n_dec_threads;
      long b = n * (idx + 1) / n_dec_threads;
      for (long i = a; i < b; i++) {
        long x = scounter_dec(s);
        assert(x <= capacity);
        assert(x > 0);
      }
    }
  }
  double t1 = cur_time();
  printf("took %.9f sec\n", t1 - t0);
  long x = scounter_get(s);
  printf("%s : value at the end = %ld\n", (x == 0? "OK" : "NG"), x);
  return (x == 0 ? 0 : 1);
}

In [ ]:
gcc -Wall -fopenmp -o tcounter_ans tcounter_ans.c

In [ ]:
./tcounter_ans 1 1 100000 1

In [ ]:
./tcounter_ans 1 1 100000 1000

In [ ]:
./tcounter_ans 10 1 100000 1000

In [ ]:
./tcounter_ans 1 10 100000 1000

In [ ]:
./tcounter_ans 10 10 1000000 1000

# <font color="green"> Problem 3 :  有限バッファの実装</font>

* 有限バッファへmutex, 条件変数を導入し, 複数のスレッドが並行に動き, capacity < n であっても正しく動くようにせよ
* 構造は, 両側飽和カウンタとよく似ているので参考にせよ

In [ ]:
%%writefile bbuf_ans.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

void die(const char * msg) {
  perror(msg); exit(1);
}

double cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_nsec * 1.0E-9 + ts->tv_sec;
}

/* 有限バッファ
   要素は必ず >= 0 とする */
typedef struct {
  long n_gets;                  /* getされた回数 */
  long n_puts;                  /* putされた回数 */
  long capacity;                /* 容量 */
  long * a;                     /* 中身(capacity要素の配列) */
  pthread_mutex_t m[1];
  pthread_cond_t gw[1];          /* get waiters */
  pthread_cond_t pw[1];          /* put waiters */
} bounded_buffer_t;

/* 容量 capacity で初期化 */
void bounded_buffer_init(bounded_buffer_t * bb, long capacity) {
  long * a = (long *)malloc(sizeof(long) * capacity);
  if (!a) die("malloc");
  bb->a = a;
  bb->capacity = capacity;
  bb->n_gets = 0;
  bb->n_puts = 0;
  pthread_mutex_init(bb->m, 0);
  pthread_cond_init(bb->gw, 0);
  pthread_cond_init(bb->pw, 0);
}

/* 要素を追加 
 * 満杯だったら待つようにするのが課題
 * 以下はそのままだと満杯の場合は0を返す(間違い)
 */
int bounded_buffer_put(bounded_buffer_t * bb, long x) {
  long cap = bb->capacity;
  pthread_mutex_lock(bb->m);
  while (1) {
    long g = bb->n_gets;
    long p = bb->n_puts;
    assert(x >= 0);
    if (p - g < cap) {
      break;
    }
    assert(p - g == cap);
    pthread_cond_wait(bb->pw, bb->m);
  }
  long g = bb->n_gets;
  long p = bb->n_puts;
  bb->a[p % cap] = x;
  bb->n_puts = p + 1;
  if (p <= g) {
    assert(p == g);
    pthread_cond_broadcast(bb->gw);
  }
  pthread_mutex_unlock(bb->m);
  return 1;                     /* OK */
}

/* 要素を取り出す
 * 空だったら待つようにするのが課題
 * 以下はそのままだと空の場合は-1を返す(間違い)
 */
long bounded_buffer_get(bounded_buffer_t * bb) {

  long cap = bb->capacity;
  pthread_mutex_lock(bb->m);
  while (1) {
    long g = bb->n_gets;
    long p = bb->n_puts;
    if (p - g > 0) {
      break;
    }
    assert(p - g == 0);
    pthread_cond_wait(bb->gw, bb->m);
  }
  long g = bb->n_gets;
  long p = bb->n_puts;
  long x = bb->a[g % cap];
  bb->n_gets = g + 1;
  assert(x >= 0);
  if (p - g >= cap) {
    assert(p - g == cap);
    pthread_cond_broadcast(bb->pw);
  }
  pthread_mutex_unlock(bb->m);
  return x;

}

int main(int argc, char ** argv) {
  long i = 1;
  /* putするスレッド数 */
  int n_putter_threads = (argc > i ? atoi(argv[i]) : 1); i++;
  /* getするスレッド数 */
  int n_getter_threads = (argc > i ? atoi(argv[i]) : 1); i++;
  /* putとgetの間にbarrierを入れるか? */
  int barrier_between_puts_gets = (argc > i ? atoi(argv[i]) : 1); i++;
  /* put (get)される回数 */
  long n                = (argc > i ? atol(argv[i]) : 1000000); i++;
  /* 容量 */
  long capacity         = (argc > i ? atol(argv[i]) : 1000); i++;
  /* 検証用(validate[x] == 1 iff getでxが取り出された) */
  char * validate = (char *)calloc(n, 1);

  bounded_buffer_t bb[1];
  bounded_buffer_init(bb, capacity);

  int nthreads = n_putter_threads + n_getter_threads;
  pthread_barrier_t barrier[1];
  pthread_barrier_init(barrier, 0, nthreads);

  double t0 = cur_time();
#pragma omp parallel num_threads(n_putter_threads + n_getter_threads)
  {
    int idx = omp_get_thread_num();
    if (idx < n_putter_threads) {
      /* I am a putter thread */
      long a = n *  idx      / n_putter_threads;
      long b = n * (idx + 1) / n_putter_threads;
      /* 0,1,...,n-1 を1つずつput */
      for (long x = a; x < b; x++) {
        int ok = bounded_buffer_put(bb, x);
        assert(ok);
      }
      if (barrier_between_puts_gets) {
        /* putが全員終わってからget */
        pthread_barrier_wait(barrier);
      }
    } else {
      /* I am a getter thread */
      idx -= n_putter_threads;
      long a = n *  idx      / n_getter_threads;
      long b = n * (idx + 1) / n_getter_threads;
      if (barrier_between_puts_gets) {
        /* putが全員終わってからget */
        pthread_barrier_wait(barrier);
      }
      /* 合計n回get */
      for (long x = a; x < b; x++) {
        long x = bounded_buffer_get(bb);
        assert(x >= 0);
        assert(x < n);
        assert(validate[x] == 0);
        validate[x] = 1;
      }
    }
  }
  double t1 = cur_time();
  printf("%f sec\n", t1 - t0);
  for (long i = 0; i < n; i++) {
    assert(validate[i]);
  }
  printf("OK\n");
  return 0;
}

In [ ]:
gcc -Wall -fopenmp -o bbuf_ans bbuf_ans.c

* 色々なパラメータで実行してみよ

In [ ]:
./bbuf_ans 1 1 1 10 10

* 以下でテストせよ

* putとgetが並行

In [ ]:
./bbuf_ans 1 1 0 100000 100

* 多数のput/getが並行

In [ ]:
./bbuf_ans 10 20 0 100000 100

In [ ]:
./bbuf_ans 20 10 0 100000 100

* 極端にcapacityが少ない

In [ ]:
./bbuf_ans 20 30 0 100000 1

In [ ]:
./bbuf_ans 30 20 0 100000 1